# 🎙️ PrecisionVoice - Vietnamese Speech-to-Text

Notebook đơn giản để transcribe audio tiếng Việt sử dụng **faster-whisper** và **pyannote** (diarization).

### Hướng dẫn
1. **Chọn GPU**: `Runtime` → `Change runtime type` → **T4 GPU**
2. **Cài đặt Secrets**: Thêm `HF_TOKEN` vào Colab Secrets (Key icon bên trái) để dùng Pyannote.
3. **Chạy từng cell** theo thứ tự từ trên xuống
4. **Sử dụng Gradio link** ở cell cuối để truy cập UI

In [ ]:
# @title 1. 🔍 Kiểm tra GPU
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✅ GPU Detected: {gpu_name}")
    print(f"   VRAM: {gpu_mem:.1f} GB")
else:
    print("⚠️ KHÔNG TÌM THẤY GPU!")
    print("👉 Vào Runtime → Change runtime type → T4 GPU")

In [ ]:
# @title 2. 📦 Cài đặt Dependencies
print("Installing dependencies...")
!pip install --upgrade torch torchvision torchaudio "pyannote.audio>=3.3.1" faster-whisper gradio librosa nest_asyncio lightning torchmetrics
!apt-get install -y -qq ffmpeg > /dev/null 2>&1
print("✅ Dependencies installed successfully!")

In [ ]:
# @title 3. 🤖 Load Models (Whisper & Pyannote)
import torch
import time
import os
import librosa
import numpy as np
from google.colab import userdata
from faster_whisper import WhisperModel
from pyannote.audio import Pipeline

try:
    from pyannote.audio.core.task import Specifications, Problem, Resolution
    torch.serialization.add_safe_globals([Specifications, Problem, Resolution])
except Exception as e:
    print(f"Could not add custom globals: {e}")

device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if device == "cuda" else "int8"

# Danh sách các model Whisper hỗ trợ
AVAILABLE_MODELS = {
    "EraX-WoW-Turbo (Whisper Large V3 Turbo - Tiếng Việt)": "erax-ai/EraX-WoW-Turbo-V1.1-CT2",
    "PhoWhisper Large (Tiếng Việt)": "kiendt/PhoWhisper-large-ct2"
}

# Cache models
loaded_whisper_models = {}
diarization_pipeline = None

# Lấy HF_TOKEN
try:
    hf_token = userdata.get('HF_TOKEN')
except:
    hf_token = os.environ.get('HF_TOKEN')

# ==================== LOAD ALL WHISPER MODELS ====================
print("="*50)
print("🔄 Pre-downloading ALL Whisper Models...")
print("="*50)

total_start = time.time()
for model_name, model_path in AVAILABLE_MODELS.items():
    print(f"\n📥 Loading: {model_name}")
    start = time.time()
    try:
        model = WhisperModel(
            model_path,
            device=device,
            compute_type=compute_type
        )
        loaded_whisper_models[f"{model_name}_{compute_type}"] = model
        print(f"   ✅ Loaded in {time.time() - start:.1f}s")
    except Exception as e:
        print(f"   ❌ Failed to load: {e}")

print(f"\n✅ All models loaded in {time.time() - total_start:.1f}s")
print(f"   Total models: {len(loaded_whisper_models)}")
print(f"   Device: {device}, Compute: {compute_type}")

# ==================== LOAD PYANNOTE ====================
print("\n" + "="*50)
print("🔄 Loading Pyannote Diarization...")
print("="*50)

if not hf_token:
    print("⚠️ WARNING: HF_TOKEN not found!")
    print("   Diarization will be disabled.")
    print("   Please set HF_TOKEN in Colab Secrets.")
else:
    start = time.time()
    try:
        diarization_pipeline = Pipeline.from_pretrained(
            "pyannote/speaker-diarization-community-1",
            token=hf_token
        )
        diarization_pipeline.to(torch.device(device))
        print(f"✅ Pyannote loaded in {time.time() - start:.1f}s")
    except Exception as e:
        print(f"❌ Failed to load Pyannote: {e}")

print("\n" + "="*50)
print("🎉 All models loaded successfully!")
print("="*50)


In [ ]:
# @title 4. 🛠️ Utilities & Helpers
import gradio as gr
import time
import nest_asyncio
import subprocess
import os

nest_asyncio.apply()

def convert_audio_to_wav(audio_path):
    """Chuẩn hóa audio về định dạng WAV 16kHz Mono."""
    try:
        # Tạo file tạm
        output_path = "temp_processed_audio.wav"
        
        # Xóa file cũ nếu tồn tại
        if os.path.exists(output_path):
            os.remove(output_path)
            
        # Command line ffmpeg
        # -i input: file đầu vào
        # -ar 16000: Sample rate 16k
        # -ac 1: Mono channel (Pyannote tốt nhất với mono)
        # -y: Overwrite output
        command = [
            "ffmpeg", 
            "-i", audio_path,
            "-ar", "16000",
            "-ac", "1",
            "-y",
            output_path
        ]
        
        subprocess.run(command, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return output_path
    except Exception as e:
        print(f"Error converting audio: {e}")
        # Fallback: Trả về file gốc nếu convert lỗi (dù rủi ro)
        return audio_path

def load_whisper_model(model_name, comp_type):
    """Dynamic load Whisper model với cache"""
    global loaded_whisper_models
    cache_key = f"{model_name}_{comp_type}"
    
    if cache_key in loaded_whisper_models:
        return loaded_whisper_models[cache_key]
    
    model_path = AVAILABLE_MODELS[model_name]
    print(f"Loading {model_name}...")
    start = time.time()
    
    model = WhisperModel(
        model_path,
        device=device,
        compute_type=comp_type
    )
    
    loaded_whisper_models[cache_key] = model
    print(f"✅ Loaded in {time.time() - start:.1f}s")
    return model

def format_timestamp(seconds):
    """Format seconds to MM:SS.ms"""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = seconds % 60
    if hours > 0:
        return f"{hours:02d}:{minutes:02d}:{secs:05.2f}"
    return f"{minutes:02d}:{secs:05.2f}"

def assign_speaker_to_segment(seg_start, seg_end, diarization_result):
    """Gán speaker cho segment dựa trên tỷ lệ overlap >= 30%."""
    if diarization_result is None:
        return "SPEAKER_00"
    
    seg_duration = seg_end - seg_start
    if seg_duration <= 0:
        return "SPEAKER_00"
    
    speaker_overlaps = {}
    
    for turn, _, speaker in diarization_result.speaker_diarization.itertracks(yield_label=True):
        overlap_start = max(seg_start, turn.start)
        overlap_end = min(seg_end, turn.end)
        overlap = max(0, overlap_end - overlap_start)
        
        if overlap > 0:
            if speaker not in speaker_overlaps:
                speaker_overlaps[speaker] = 0
            speaker_overlaps[speaker] += overlap
    
    if not speaker_overlaps:
        return "SPEAKER_00"
    
    best_speaker = max(speaker_overlaps, key=speaker_overlaps.get)
    best_overlap = speaker_overlaps[best_speaker]
    
    if best_overlap / seg_duration >= 0.3:
        return best_speaker
    
    return "SPEAKER_00"

def merge_consecutive_segments(segments, max_gap=0.5):
    """Gộp các segment liên tiếp của cùng một speaker."""
    if not segments:
        return []
    
    merged = []
    current = segments[0].copy()
    
    for seg in segments[1:]:
        if seg['speaker'] == current['speaker'] and (seg['start'] - current['end']) <= max_gap:
            current['end'] = seg['end']
            current['text'] += ' ' + seg['text']
        else:
            merged.append(current)
            current = seg.copy()
    
    merged.append(current)
    return merged

In [ ]:
# @title 5. ⚙️ Processing Logic
def process_audio(audio_path, model_name, language, beam_size, vad_filter, vad_min_silence, vad_speech_pad, vad_min_speech, vad_threshold, temperature, best_of, patience, length_penalty, initial_prompt, prefix, condition_on_previous_text, no_speech_threshold, log_prob_threshold, compression_ratio_threshold, comp_type, merge_segs, p=gr.Progress()):
    """
    Quy trình mới:
    0. Chuẩn hóa audio (convert mp3 -> wav 16k).
    1. Diarization để tách các đoạn của từng người nói.
    2. Cắt audio theo các đoạn này.
    3. Transcribe từng đoạn audio.
    4. Gộp kết quả.
    """
    if audio_path is None:
        msg = "⚠️ Vui lòng upload hoặc ghi âm audio!"
        return msg, msg
    
    total_start_time = time.time()
    
    # Check Pyannote
    if diarization_pipeline is None:
        return "❌ Lỗi: Chưa load được Pyannote (kiểm tra HF_TOKEN).", "❌ Lỗi: Chưa load được Pyannote."

    # 0. Preprocessing Audio (Standardize)
    p(0.05, desc="Đang chuẩn hóa audio (16kHz WAV)...")
    try:
        # Luôn convert về wav 16k mono để tránh lỗi sample rate mismatch của Pyannote
        clean_audio_path = convert_audio_to_wav(audio_path)
    except Exception as e:
        msg = f"❌ Lỗi convert audio: {e}"
        return msg, msg
        
    # 1. Load Standardized Audio for slicing later
    p(0.08, desc="Đang đọc file audio...")
    try:
        y, sr = librosa.load(clean_audio_path, sr=16000)
        # sr should be 16000 now exactly
    except Exception as e:
        return f"❌ Lỗi đọc audio: {e}", f"❌ Lỗi đọc audio: {e}"

    # 2. DIARIZATION
    p(0.1, desc="Đang phân tách người nói (Diarization)...")
    
    try:
        # Sử dụng file đã chuẩn hóa
        diarization = diarization_pipeline(clean_audio_path)
    except Exception as e:
        return f"❌ Lỗi Diarization: {e}", f"❌ Lỗi Diarization: {e}"
        
    diarization_segments = []
    # Dùng cách user đã fix trước đó (nếu model trả về object khác)
    # Mặc định pipeline community trả về Annotation trực tiếp, nhưng user fix thành diarization.speaker_diarization
    # Mình sẽ try/except để support cả 2 structure cho an toàn
    try:
        # Trường hợp 1: Standard Annotation
        iterator = diarization.itertracks(yield_label=True)
        # Test thử xem có chạy ko, nếu không phải Annotation nó sẽ lỗi attribute
        _ = list(iterator)
        # Reset iterate
        iterator = diarization.itertracks(yield_label=True)
    except:
        # Trường hợp 2: User report structure (maybe wrapper)
        try:
             iterator = diarization.speaker_diarization.itertracks(yield_label=True)
        except:
             return "❌ Lỗi format result Diarization", "❌ Lỗi format result Diarization"

    for turn, _, speaker in iterator:
        diarization_segments.append({
            "start": turn.start,
            "end": turn.end,
            "speaker": speaker
        })
    
    # Sort segments by start time
    diarization_segments.sort(key=lambda x: x['start'])
    
    # Merge consecutive segments if requested
    if merge_segs and diarization_segments:
        p(0.3, desc="Đang gộp segment liên tiếp...")
        merged = []
        current = diarization_segments[0].copy()
        for seg in diarization_segments[1:]:
            if seg['speaker'] == current['speaker'] and (seg['start'] - current['end']) <= 0.5:
                current['end'] = seg['end']
            else:
                merged.append(current)
                current = seg.copy()
        merged.append(current)
        diarization_segments = merged
    
    # 3. TRANSCRIPTION LOOP
    p(0.4, desc="Đang tải model Whisper...")
    model = load_whisper_model(model_name, comp_type)
    
    processed_segments = []
    
    total_segs = len(diarization_segments)
    
    # Prepare VAD options
    if vad_filter:
        vad_options = dict(
            min_silence_duration_ms=vad_min_silence,
            speech_pad_ms=vad_speech_pad,
            min_speech_duration_ms=vad_min_speech,
            threshold=vad_threshold
        )
    else:
        vad_options = False
        
    prompt = initial_prompt.strip() if (initial_prompt and initial_prompt.strip()) else None
    prefix_text = prefix.strip() if (prefix and prefix.strip()) else None

    print(f"Processing {total_segs} segments...")
    
    for idx, seg in enumerate(diarization_segments):
        start_sec = seg['start']
        end_sec = seg['end']
        speaker = seg['speaker']
        
        # UI Progress
        progress_val = 0.4 + (0.5 * (idx / total_segs))
        p(progress_val, desc=f"Transcribing {idx+1}/{total_segs} ({speaker})...")
        
        # Audio slicing
        start_sample = int(start_sec * sr)
        end_sample = int(end_sec * sr)
        
        # Avoid empty slice
        if end_sample <= start_sample:
            continue
            
        y_seg = y[start_sample:end_sample]
        
        # Whisper Transcribe for this chunk
        try:
            # Note: We pass the numpy array 'y_seg' directly
            segments_gen, _ = model.transcribe(
                y_seg, 
                language=language if language != "auto" else None,
                beam_size=beam_size, 
                vad_filter=vad_options,
                temperature=temperature,
                best_of=best_of,
                patience=patience,
                length_penalty=length_penalty,
                initial_prompt=prompt,
                prefix=prefix_text,
                condition_on_previous_text=condition_on_previous_text,
                no_speech_threshold=no_speech_threshold,
                log_prob_threshold=log_prob_threshold,
                compression_ratio_threshold=compression_ratio_threshold,
                word_timestamps=False 
            )
            
            # Collect text
            seg_text_parts = []
            for s in segments_gen:
                seg_text_parts.append(s.text.strip())
            
            final_text = " ".join(seg_text_parts).strip()
            
            if final_text:
                # Store Result
                processed_segments.append({
                    "start": start_sec,
                    "end": end_sec,
                    "speaker": speaker,
                    "text": final_text
                })
                
        except Exception as e:
            print(f"Error transcribing segment {idx}: {e}")
            continue

    total_elapsed = time.time() - total_start_time
    
    p(0.95, desc="Đang xuất kết quả...")
    
    # ========== OUTPUT GENERATION ==========
    
    # Speaker colors
    speaker_colors = {
        'SPEAKER_00': '🔵',
        'SPEAKER_01': '🟢', 
        'SPEAKER_02': '🟡',
        'SPEAKER_03': '🟠',
        'SPEAKER_04': '🔴',
        'SPEAKER_05': '🟣',
    }
    
    # 1. Plain Transcription Output
    transcribe_lines = []
    for item in processed_segments:
        ts = f"[{format_timestamp(item['start'])} → {format_timestamp(item['end'])}]"
        transcribe_lines.append(f"{ts} {item['text']}")
        
    transcribe_header = f"""## 📝 Kết quả Transcription

| Thông tin | Giá trị |
|-----------|----------|
| ⏱️ Tổng thời gian xử lý | {total_elapsed:.1f}s |
| 📊 Tổng số Segment | {len(processed_segments)} |

---

"""
    transcribe_output = transcribe_header + "\n".join(transcribe_lines)
    
    # 2. Diarization + Transcription Output
    diarize_lines = []
    unique_speakers = set()
    
    for item in processed_segments:
        unique_speakers.add(item['speaker'])
        ts = f"[{format_timestamp(item['start'])} → {format_timestamp(item['end'])}]"
        icon = speaker_colors.get(item['speaker'], '⚪')
        diarize_lines.append(f"{ts} {icon} **{item['speaker']}**: {item['text']}")
        
    diarize_header = f"""## 🎭 Kết quả Transcription + Diarization

| Thông tin | Giá trị |
|-----------|----------|
| 👥 Số người nói | {len(unique_speakers)} |
| ⏱️ Tổng thời gian xử lý | {total_elapsed:.1f}s |
| 📊 Tổng số Segment | {len(processed_segments)} |

---

"""
    diarize_output = diarize_header + "\n".join(diarize_lines)
    
    return transcribe_output, diarize_output

In [ ]:
# @title 6. 🚀 Gradio UI
css = """
.gradio-container { max-width: 1200px !important; }
.output-markdown { font-family: 'JetBrains Mono', monospace !important; }
"""

with gr.Blocks(title="PrecisionVoice", theme=gr.themes.Soft(), css=css) as demo:
    gr.Markdown("""# 🎙️ PrecisionVoice - Vietnamese Speech-to-Text
    
Sử dụng **Whisper** để nhận dạng văn bản và **Pyannote** để phân biệt người nói.
""")
    
    with gr.Row():
        with gr.Column(scale=1):
            audio_input = gr.Audio(
                sources=["upload", "microphone"], 
                type="filepath", 
                label="🔊 Audio Input"
            )
            
            gr.Markdown("### ⚙️ Cài đặt Model")
            model_select = gr.Dropdown(
                choices=list(AVAILABLE_MODELS.keys()),
                value=list(AVAILABLE_MODELS.keys())[0],
                label="🤖 Whisper Model"
            )
            
            language = gr.Dropdown(
                choices=["auto", "vi", "en", "zh", "ja", "ko"],
                value="vi",
                label="🌐 Ngôn ngữ"
            )
            
            comp_type_select = gr.Dropdown(
                choices=["float16", "float32", "int8", "int8_float16"],
                value=compute_type,
                label="⚡ Compute Type"
            )
            
            with gr.Accordion("🔧 Tùy chọn nâng cao", open=False):
                beam_size = gr.Slider(
                    minimum=1, maximum=10, value=5, step=1,
                    label="Beam Size",
                    info="Cao hơn = chính xác hơn nhưng chậm hơn"
                )
                vad_filter = gr.Checkbox(
                    value=True, 
                    label="VAD Filter",
                    info="Lọc khoảng lặng tự động"
                )
                with gr.Row():
                    vad_min_silence = gr.Number(value=1000, label="Min Silence (ms)", info="min_silence_duration_ms")
                    vad_speech_pad = gr.Number(value=400, label="Speech Pad (ms)", info="speech_pad_ms")
                with gr.Row():
                    vad_min_speech = gr.Number(value=250, label="Min Speech (ms)", info="min_speech_duration_ms")
                    vad_threshold = gr.Slider(minimum=0, maximum=1, value=0.5, step=0.05, label="VAD Threshold")
            
            with gr.Accordion("🧠 Tham số Generation (Whisper)", open=False):
                with gr.Row():
                    temperature = gr.Slider(0.0, 1.0, value=0.0, step=0.1, label="Temperature")
                    best_of = gr.Number(value=5, label="Best Of")
                with gr.Row():
                    patience = gr.Number(value=1.0, label="Patience", step=0.1)
                    length_penalty = gr.Number(value=1.0, label="Length Penalty", step=0.1)
                initial_prompt = gr.Textbox(label="Initial Prompt", placeholder="Ngữ cảnh hoặc từ vựng...")
                prefix = gr.Textbox(label="Prefix", placeholder="Bắt đầu câu với...")
                condition_on_previous_text = gr.Checkbox(value=True, label="Condition on previous text")
                
                gr.Markdown("**Filter Thresholds**")
                with gr.Row():
                    no_speech_threshold = gr.Slider(0.0, 1.0, value=0.6, step=0.05, label="No Speech Threshold")
                    log_prob_threshold = gr.Slider(-5.0, 0.0, value=-1.0, step=0.1, label="Log Prob Threshold")
                    compression_ratio_threshold = gr.Number(value=2.4, label="Compression Ratio Threshold")
            
            merge_segments = gr.Checkbox(
                value=True,
                label="Gộp Segment cùng Speaker",
                info="Gộp các câu liên tiếp của cùng người nói"
            )
            
            btn_process = gr.Button("🚀 Xử lý Audio", variant="primary", size="lg")
        
        with gr.Column(scale=2):
            with gr.Tabs():
                with gr.Tab("📝 Transcription"):
                    output_transcribe = gr.Markdown(
                        value="*Kết quả transcription sẽ hiển thị ở đây...*",
                        elem_classes=["output-markdown"]
                    )
                with gr.Tab("🎭 Transcription + Diarization"):
                    output_diarize = gr.Markdown(
                        value="*Kết quả transcription + diarization sẽ hiển thị ở đây...*",
                        elem_classes=["output-markdown"]
                    )
    
    btn_process.click(
        process_audio,
        inputs=[
            audio_input, model_select, language, beam_size, vad_filter, 
            vad_min_silence, vad_speech_pad, vad_min_speech, vad_threshold,
            temperature, best_of, patience, length_penalty, 
            initial_prompt, prefix, condition_on_previous_text,
            no_speech_threshold, log_prob_threshold, compression_ratio_threshold,
            comp_type_select, merge_segments
        ],
        outputs=[output_transcribe, output_diarize]
    )
    
    gr.Markdown("""---
    
### 📖 Hướng dẫn sử dụng

1. **Upload audio** hoặc ghi âm trực tiếp
2. **Chọn Model**:
   - `EraX-WoW-Turbo`: Whisper Large V3 Turbo, tối ưu cho tiếng Việt
   - `PhoWhisper Large`: Model được huấn luyện riêng cho tiếng Việt
3. **Setting nâng cao**:
   - Chỉnh `temperature` nếu muốn model sáng tạo hơn.
   - Thêm `Initial Prompt` để gợi ý từ vựng chuyên ngành.
4. **Nhấn "🚀 Xử lý Audio"** để nhận kết quả ở cả 2 tab
""")

# Launch
import os
if "COLAB_GPU" in os.environ or "google.colab" in str(get_ipython()):
    demo.queue().launch(share=True, debug=True)
else:
    demo.launch(share=False)